In [1]:
from neo4j import GraphDatabase
import pandas as pd
import ast

In [ ]:
uri      = "bolt://localhost:7687"
username = "neo4j"
password = "linasadgal"
driver   = GraphDatabase.driver(uri, auth=(username, password))

In [11]:
df = pd.read_csv('movies.csv')
df.shape

(199, 6)

In [13]:
df.sample(5)

,movie_title,release_year,genres,actors,director,emotions_conveyed
90,Time Out,2023,['Drama'],"['Karim Leklou', 'Isaka Sawadogo']",Ève Duchemin,"['Isolation', 'Hope', 'Emotional Turmoil']"
125,The Woman King,2022,"['Action', 'Drama', 'History']","['Viola Davis', 'Thuso Mbedu']",Gina Prince-Bythewood,"['Empowerment', 'Courage', 'Sacrifice']"
99,Love Life,2023,['Drama'],"['Fumino Kimura', 'Kento Nagayama']",Kōji Fukada,"['Love', 'Loss', 'Resilience']"
27,Parallel Earth,2025,"['Sci-Fi', 'Thriller', 'Drama']","['Matt Damon', 'Julianne Moore', 'Timothée Cha...",'Christopher Nolan',"['Suspense', 'Mystery', 'Existential']"
115,John Wick: Chapter 4,2023,"['Action', 'Crime', 'Thriller']",['Keanu Reeves'],Chad Stahelski,"['Vengeance', 'Honor', 'Tension']"


In [5]:
def create_constraints():
    with driver.session() as session:
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Movie)     REQUIRE m.title      IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (a:Actor)     REQUIRE a.name       IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Director)  REQUIRE d.name       IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (g:Genre)     REQUIRE g.name       IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (e:Emotion)   REQUIRE e.name       IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (r:ReleaseDate) REQUIRE r.year     IS UNIQUE")


create_constraints()


In [17]:
# helper to parse your list‑strings 
def parse_list_str(s: str) -> list:
    """
    Turn Python‑list‑looking strings (with single quotes) into a real list,
    even if items contain apostrophes.
    """
    s = s.strip()
    # remove [ and ]
    if len(s) >= 2 and s[0] == "[" and s[-1] == "]":
        inner = s[1:-1].strip()
    else:
        inner = s

    if not inner:
        return []

    # split on the only safe boundary "', '"
    parts = inner.split("', '")
    cleaned = []
    for i, p in enumerate(parts):
        # first element may start with a quote
        if i == 0 and p.startswith("'"):
            p = p[1:]
        # last element may end with a quote
        if i == len(parts)-1 and p.endswith("'"):
            p = p[:-1]
        cleaned.append(p)
    return cleaned


In [19]:
def import_movies(csv_path: str):
    df = pd.read_csv(csv_path)
    for col in ['genres', 'actors', 'emotions_conveyed']:
        df[col] = df[col].astype(str).apply(parse_list_str)

    with driver.session() as session:
        for row in df.itertuples(index=False):
            title, year, genres, actors, director, emotions = (
                row.movie_title,
                int(row.release_year),
                row.genres,
                row.actors,
                row.director,
                row.emotions_conveyed
            )

            session.run(
                """
                MERGE (m:Movie {title: $title})
                SET m.year = $year
                WITH m
                MERGE (d:Director {name: $director})
                MERGE (m)-[:DIRECTED_BY]->(d)
                WITH m
                MERGE (r:ReleaseDate {year: $year})
                MERGE (m)-[:RELEASED_ON]->(r)
                """,
                {"title": title, "year": year, "director": director}
            )

            for g in genres:
                session.run(
                    """
                    MERGE (g:Genre {name: $g})
                    WITH g
                    MATCH (m:Movie {title: $title})
                    MERGE (m)-[:IN_GENRE]->(g)
                    """,
                    {"g": g, "title": title}
                )

            for a in actors:
                session.run(
                    """
                    MERGE (a:Actor {name: $a})
                    WITH a
                    MATCH (m:Movie {title: $title})
                    MERGE (m)-[:ACTED_IN]->(a)
                    """,
                    {"a": a, "title": title}
                )

            for e in emotions:
                session.run(
                    """
                    MERGE (e:Emotion {name: $e})
                    WITH e
                    MATCH (m:Movie {title: $title})
                    MERGE (m)-[:CONVEYS]->(e)
                    """,
                    {"e": e, "title": title}
                )


In [21]:
import_movies("movies.csv")

In [ ]:
driver.close()